In [3]:
import torch
from torch.utils.data.dataset import Dataset
import transformers
import tokenizers
from tokenizers import Tokenizer, pre_tokenizers
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace, Split
import json
from datasets import load_dataset, load_dataset_builder, get_dataset_split_names,get_dataset_config_names
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import BertConfig, BertLMHeadModel
from transformers import Trainer, TrainingArguments
from tokenizers import ByteLevelBPETokenizer
import numpy as np
import pandas as pd

In [2]:
from datasets import load_dataset
dataset = load_dataset("code_x_glue_cc_clone_detection_big_clone_bench")

Found cached dataset code_x_glue_cc_clone_detection_big_clone_bench (/home/aktarafder/.cache/huggingface/datasets/code_x_glue_cc_clone_detection_big_clone_bench/default/0.0.0/4c85673dc4ced396531cf81e6c10e61f23eca84ca840ca137f52a15dd19c8789)
100%|██████████| 3/3 [00:00<00:00, 77.96it/s]


In [7]:
dataset['test']['label'][4]

False

In [13]:
dataset['test'].shape

(415416, 6)

In [42]:
f=0
t=0
i=0
false_clones=[]
true_clones=[]

while(True):
    if (f<100):
        if (dataset['train']['label'][i]==False):
            f=f+1
            false_clones.append(i)
        else:
            t=t+1
        i=i+1
    else:
        break

print("number of false clones collected: ",f)
print(false_clones)

number of false clones collected:  100
[0, 3, 4, 5, 6, 9, 13, 14, 16, 19, 20, 21, 22, 23, 25, 26, 36, 37, 39, 41, 44, 45, 46, 47, 51, 52, 54, 56, 59, 61, 71, 74, 77, 79, 80, 83, 86, 87, 88, 89, 90, 91, 94, 99, 100, 101, 103, 104, 105, 106, 107, 109, 110, 111, 116, 118, 119, 122, 124, 125, 126, 127, 128, 129, 132, 133, 135, 138, 140, 143, 144, 147, 151, 153, 155, 158, 159, 161, 167, 169, 170, 175, 179, 183, 184, 185, 188, 189, 190, 191, 192, 196, 197, 198, 200, 201, 202, 204, 205, 206]


false_clone_indices=
[0, 3, 4, 5, 6, 9, 13, 14, 16, 19, 20, 21, 22, 23, 25, 26, 36, 37, 39, 41, 44, 45, 46, 47, 51, 52, 54, 56, 59, 61, 71, 74, 77, 79, 80, 83, 86, 87, 88, 89, 90, 91, 94, 99, 100, 101, 103, 104, 105, 106, 107, 109, 110, 111, 116, 118, 119, 122, 124, 125, 126, 127, 128, 129, 132, 133, 135, 138, 140, 143, 144, 147, 151, 153, 155, 158, 159, 161, 167, 169, 170, 175, 179, 183, 184, 185, 188, 189, 190, 191, 192, 196, 197, 198, 200, 201, 202, 204, 205, 206]

In [41]:
t=0
i=0
true_clones=[]

while(True):
    if (t<100):
        if (dataset['train']['label'][i]==True):
            t=t+1
            true_clones.append(i)
            print(t,end=' ')
        else:
            f=f+1
        i=i+1
    else:
        break

print("number of true clones collected: ",t)
print(true_clones)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 number of true clones collected:  100
[1, 2, 7, 8, 10, 11, 12, 15, 17, 18, 24, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 40, 42, 43, 48, 49, 50, 53, 55, 57, 58, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 75, 76, 78, 81, 82, 84, 85, 92, 93, 95, 96, 97, 98, 102, 108, 112, 113, 114, 115, 117, 120, 121, 123, 130, 131, 134, 136, 137, 139, 141, 142, 145, 146, 148, 149, 150, 152, 154, 156, 157, 160, 162, 163, 164, 165, 166, 168, 171, 172, 173, 174, 176, 177, 178, 180, 181, 182]


true_clone_indices=
[1, 2, 7, 8, 10, 11, 12, 15, 17, 18, 24, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 40, 42, 43, 48, 49, 50, 53, 55, 57, 58, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 75, 76, 78, 81, 82, 84, 85, 92, 93, 95, 96, 97, 98, 102, 108, 112, 113, 114, 115, 117, 120, 121, 123, 130, 131, 134, 136, 137, 139, 141, 142, 145, 146, 148, 149, 150, 152, 154, 156, 157, 160, 162, 163, 164, 165, 166, 168, 171, 172, 173, 174, 176, 177, 178, 180, 181, 182]

In [52]:
false_clones_strings=[]
for f in false_clones:
    fpairs=[]
    fpairs.append(dataset['train']['func1'][f])
    fpairs.append(dataset['train']['func2'][f])
    false_clones_strings.append(fpairs)

In [55]:
false_clones_strings

[['    private void setNodekeyInJsonResponse(String service) throws Exception {\n        String filename = this.baseDirectory + service + ".json";\n        Scanner s = new Scanner(new File(filename));\n        PrintWriter fw = new PrintWriter(new File(filename + ".new"));\n        while (s.hasNextLine()) {\n            fw.println(s.nextLine().replaceAll("NODEKEY", this.key));\n        }\n        s.close();\n        fw.close();\n        (new File(filename + ".new")).renameTo(new File(filename));\n    }\n',
  '    public void transform(String style, String spec, OutputStream out) throws IOException {\n        URL url = new URL(rootURL, spec);\n        InputStream in = new PatchXMLSymbolsStream(new StripDoctypeStream(url.openStream()));\n        transform(style, in, out);\n        in.close();\n    }\n'],
 ['    public static void printResponseHeaders(String address) {\n        logger.info("Address: " + address);\n        try {\n            URL url = new URL(address);\n            URLConne

In [67]:
# Import pandas library
import pandas as pd
 
# Create the pandas DataFrame
falseClonesdf = pd.DataFrame(false_clones_strings, columns = ['clone1', 'clone2'])
falseClonesdf.to_csv('falseClonesdf.csv', index=False)
print(falseClonesdf )

                                               clone1  \
0       private void setNodekeyInJsonResponse(Stri...   
1       public static void printResponseHeaders(St...   
2       public void load(String fileName) {\n     ...   
3       private MapProperties readProperties(URL u...   
4       protected void doRestoreOrganize() throws ...   
..                                                ...   
95      private void loadDefaultDrivers() {\n     ...   
96      public void deletePortletName(PortletName ...   
97      protected String getPostRequestContent(Str...   
98      public static void main(String[] args) {\n...   
99      public JarClassLoader(ClassLoader parent) ...   

                                               clone2  
0       public void transform(String style, String...  
1       public static String getEncodedPassword(St...  
2       private static void copyFile(File sourceFi...  
3       public String tranportRemoteUnitToLocalTem...  
4       static String encodeEmailAs

In [62]:
true_clones=[1, 2, 7, 8, 10, 11, 12, 15, 17, 18, 24, 27, 28, 29, 30, 31, 32, 33, 34, 35, 38, 40, 42, 43, 48, 49, 50, 53, 55, 57, 58, 60, 62, 63, 64, 65, 66, 67, 68, 69, 70, 72, 73, 75, 76, 78, 81, 82, 84, 85, 92, 93, 95, 96, 97, 98, 102, 108, 112, 113, 114, 115, 117, 120, 121, 123, 130, 131, 134, 136, 137, 139, 141, 142, 145, 146, 148, 149, 150, 152, 154, 156, 157, 160, 162, 163, 164, 165, 166, 168, 171, 172, 173, 174, 176, 177, 178, 180, 181, 182]

In [63]:
true_clones_strings=[]
for t in true_clones:
    tpairs=[]
    tpairs.append(dataset['train']['func1'][t])
    tpairs.append(dataset['train']['func2'][t])
    true_clones_strings.append(tpairs)

In [66]:
trueClonesdf = pd.DataFrame(true_clones_strings, columns = ['clone1', 'clone2'])
trueClonesdf.to_csv('trueClonesdf.csv', index=False)
print(trueClonesdf )

                                               clone1  \
0       public static void test(String args[]) {\n...   
1       public String kodetu(String testusoila) {\...   
2       private String sha1(String s) {\n        S...   
3       private void storeFieldMap(WorkingContent ...   
4       private static void main(String[] args) {\...   
..                                                ...   
95      public void convert(File src, File dest) t...   
96      public void copyFileFromLocalMachineToRemo...   
97      private static void insertModuleInEar(File...   
98      public static void main(String argv[]) {\n...   
99      public static int zipFile(File file_input,...   

                                               clone2  
0       private static String loadUrlToString(Stri...  
1       private StringBuffer encoder(String arg) {...  
2       @SuppressWarnings("unused")\n    private S...  
3       public void elimina(Pedido pe) throws erro...  
4       public static void copyFile

[]

# Comparison Code

In [92]:
from transformers import PreTrainedTokenizerFast
from transformers import BertConfig, BertLMHeadModel


calculating similarities

working with ast first, saving it and then moving onto nonast. Just change the tokenizer and model directory

In [93]:
tokenizer=Tokenizer.from_file('./ASTbpeTokens1to25k_Ast_java')
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
fast_tokenizer.mask_token='<mask>'
fast_tokenizer.pad_token='<pad>'


In [94]:
model = BertLMHeadModel.from_pretrained('./ast_transformer_ast_no_pretrain/checkpoint-15000/',output_hidden_states=True)

In [95]:
# loading the clone pairs
tdf=pd.read_csv('trueClonesdf.csv')

In [96]:
model

BertLMHeadModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50000, 30, padding_idx=0)
      (position_embeddings): Embedding(512, 30)
      (token_type_embeddings): Embedding(2, 30)
      (LayerNorm): LayerNorm((30,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=30, out_features=30, bias=True)
              (key): Linear(in_features=30, out_features=30, bias=True)
              (value): Linear(in_features=30, out_features=30, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=30, out_features=30, bias=True)
              (LayerNorm): LayerNorm((30,), eps=1e-12, elementwise_affine=True)
         

In [97]:
clone1_embed=np.zeros((100,30))
clone2_embed=np.zeros((100,30))
clone1_embed.shape;
clone2_embed.shape
euclidean_dist=np.zeros(100)
cosine_dist=np.zeros(100)
manhattan_dis=np.zeros(100)
dot_product=np.zeros(100)

In [98]:
tdf['clone1'][10]==tdf['clone2'][10]

False

In [99]:
with torch.no_grad():
    for i in range(100):
        
        code1 = fast_tokenizer.encode(tdf['clone1'][i],max_length=512, return_tensors='pt')
        code2 = fast_tokenizer.encode(tdf['clone2'][i],max_length=512, return_tensors='pt')
        out_code1 = model(code1)
        out_code2 = model(code2)

        clone1_embed[i]=out_code1.hidden_states[2].numpy().mean(axis=1)
        clone2_embed[i]=out_code2.hidden_states[2].numpy().mean(axis=1)
        
        # we are considering the last hidden layer as embedding
        # number of hidden_layer in trained model in BERT= 1+number of hidden layers

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [100]:
clone1_embed.shape

(100, 30)

In [101]:
def calculate_distance(n1, n2):
    """
    takes 2 numpy array
    n1, n2, each a vector embedding for a single code_string 
    """
    euclidean_dist=np.linalg.norm(n1-n2)
    #print(n1==n2)
    manhattan_dist=np.sum(np.abs(n1 - n2))
    cosine_sim=np.dot(n1,n2)/(np.linalg.norm(n1)*np.linalg.norm(n2))
    dot_product=np.dot(n1,n2)

    return euclidean_dist, manhattan_dist, cosine_sim, dot_product
    
def compare_dis_2models(clone1_embed, clone2_embed, number_of_pairs=100):

    euclidean_dist=np.zeros(number_of_pairs)
    cosine_sim=np.zeros(number_of_pairs)
    manhattan_dist=np.zeros(number_of_pairs)
    dot_product=np.zeros(number_of_pairs)

    for i in range(number_of_pairs):
        e,m,c,d=calculate_distance(clone1_embed[i], clone2_embed[i])
        euclidean_dist[i]=e
        manhattan_dist[i]=m
        cosine_sim[i]=c
        dot_product[i]=d

    return euclidean_dist, manhattan_dis, cosine_sim, dot_product

In [102]:
euclidean_dist, manhattan_dis, cosine_sim, dot_product=compare_dis_2models(clone1_embed, clone2_embed, number_of_pairs=100)

In [103]:
# done once for ast base and another for non ast
cosine_sim_df=pd.DataFrame(cosine_sim, columns=['distance'])
cosine_sim_df.to_csv('./distances/true_clones/cosine_sim_ast_true_clones.csv', index=False)
euclidean_df=pd.DataFrame(euclidean_dist,columns=['distance'])
euclidean_df.to_csv('./distances/true_clones/euclidean_dist_ast_true_clones.csv', index=False)
manhattan_dis_df=pd.DataFrame(manhattan_dis,columns=['distance'])
manhattan_dis_df.to_csv('./distances/true_clones/manhattan_ast_true_clones.csv', index=False)
dot_product_df=pd.DataFrame(dot_product, columns=['distance'])
dot_product_df.to_csv('./distances/true_clones/dot_product_ast_true_clones.csv', index=False)

#non ast calculations

In [104]:
tokenizer=Tokenizer.from_file('./ASTbpeTokens1to25k_no_Ast_java')
fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)
fast_tokenizer.mask_token='<mask>'
fast_tokenizer.pad_token='<pad>'


In [105]:
# loading a transformer trained on tokenizer without ast
model = BertLMHeadModel.from_pretrained('./ast_transformer_noAst_no_pretrain/checkpoint-15000/',output_hidden_states=True)

In [106]:
# doing the same calculations but here for different model

In [107]:
clone1_embed=np.zeros((100,30))
clone2_embed=np.zeros((100,30))
clone1_embed.shape;
clone2_embed.shape
euclidean_dist=np.zeros(100)
cosine_dist=np.zeros(100)
manhattan_dis=np.zeros(100)
dot_product=np.zeros(100)

In [108]:
with torch.no_grad():
    for i in range(100):
        
        code1 = fast_tokenizer.encode(tdf['clone1'][i],max_length=512, return_tensors='pt')
        code2 = fast_tokenizer.encode(tdf['clone2'][i],max_length=512, return_tensors='pt')
        out_code1 = model(code1)
        out_code2 = model(code2)

        clone1_embed[i]=out_code1.hidden_states[2].numpy().mean(axis=1)
        clone2_embed[i]=out_code2.hidden_states[2].numpy().mean(axis=1)
        
        # we are considering the last hidden layer as embedding
        # number of hidden_layer in trained model in BERT= 1+number of hidden layers

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [109]:
# for non ast 
euclidean_dist, manhattan_dis, cosine_sim, dot_product=compare_dis_2models(clone1_embed, clone2_embed, number_of_pairs=100)

In [110]:
# done once for ast base and another for non ast
cosine_sim_df=pd.DataFrame(cosine_sim, columns=['distance'])
cosine_sim_df.to_csv('./distances/true_clones/cosine_sim_No_ast_true_clones.csv', index=False)
euclidean_df=pd.DataFrame(euclidean_dist,columns=['distance'])
euclidean_df.to_csv('./distances/true_clones/euclidean_dist_No_ast_true_clones.csv', index=False)
manhattan_dis_df=pd.DataFrame(manhattan_dis,columns=['distance'])
manhattan_dis_df.to_csv('./distances/true_clones/manhattan_No_ast_true_clones.csv', index=False)
dot_product_df=pd.DataFrame(dot_product, columns=['distance'])
dot_product_df.to_csv('./distances/true_clones/dot_product_No_ast_true_clones.csv', index=False)